In [1]:
import glassppy as glass
from ann_dataset import dataset_dict

In [2]:
topk = 10
dataset = dataset_dict['sift-128-euclidean']()
X_train = dataset.get_database()
X_test = dataset.get_queries()
Y = dataset.get_groundtruth(topk)
n, d = X_train.shape
nq, d = X_test.shape
metric = dataset.metric
print(f"n = {n}, d = {d}, nq = {nq}, metric = {metric}")
print(f"dataset size = {n * d * 4 / 1024 / 1024:.2f}MB")


n = 1000000, d = 128, nq = 10000, metric = L2
dataset size = 488.28MB


In [3]:
index = glass.Index(index_type="NSG", dim=d, metric=metric, R=32, L=200)
g = index.build(X_train)


NNDescent iter: [1/10], recall: 0.000312
NNDescent iter: [2/10], recall: 0.002656
NNDescent iter: [3/10], recall: 0.023281
NNDescent iter: [4/10], recall: 0.116719
NNDescent iter: [5/10], recall: 0.323281
NNDescent iter: [6/10], recall: 0.564844
NNDescent iter: [7/10], recall: 0.741562
NNDescent iter: [8/10], recall: 0.862813
NNDescent iter: [9/10], recall: 0.920625
NNDescent iter: [10/10], recall: 0.960312
NNDescent cost: 16.76s
NSG building progress: [10000/1000000]
NSG building progress: [20000/1000000]
NSG building progress: [30000/1000000]
NSG building progress: [40000/1000000]
NSG building progress: [50000/1000000]
NSG building progress: [60000/1000000]
NSG building progress: [70000/1000000]
NSG building progress: [80000/1000000]
NSG building progress: [90000/1000000]
NSG building progress: [100000/1000000]
NSG building progress: [110000/1000000]
NSG building progress: [120000/1000000]
NSG building progress: [130000/1000000]
NSG building progress: [140000/1000000]
NSG building pr

In [4]:
s = glass.Searcher(graph=g, data=X_train, metric=metric, level=2)
s.set_ef(36)


In [5]:
from os import cpu_count
from time import time
num_threads = cpu_count()

pred = s.batch_search(query=X_test, k=topk, num_threads=num_threads).reshape(-1, topk)
recall = dataset.evaluate(pred, topk)
print(f"Recall = {recall * 100:.2f}%")


Recall = 91.58%


In [6]:
s.batch_search(query=X_test, k=topk, num_threads=num_threads)  # warmup
for iter in range(10):
    t1 = time()
    pred = s.batch_search(query=X_test, k=topk, num_threads=num_threads)
    t2 = time()
    print(f"QPS = {nq / (t2 - t1):.2f}")


QPS = 670584.36
QPS = 692083.69
QPS = 695422.88
QPS = 692838.22
QPS = 696069.17
QPS = 693973.10
QPS = 696589.38
QPS = 694869.87
QPS = 694363.71
QPS = 693239.01


In [7]:
s.optimize()


=============Start optimization=============
settint best po = 7, best pl = 1
gaining 2.42% performance improvement
=============Done optimization=============


In [8]:
s.batch_search(query=X_test, k=topk, num_threads=num_threads)  # warmup
for iter in range(10):
    t1 = time()
    pred = s.batch_search(query=X_test, k=topk,
                          num_threads=num_threads).reshape(-1, topk)
    recall = dataset.evaluate(pred, topk)
    print(recall)
    t2 = time()
    print(f"QPS = {nq / (t2 - t1)}")


0.91584
QPS = 42312.34212204118
0.91584
QPS = 41535.45260360087
0.91584
QPS = 41688.40720876884
0.91584
QPS = 41218.09612534739
0.91584
QPS = 41950.12957189765
0.91584
QPS = 41334.024483263594
0.91584
QPS = 41325.79788282217
0.91584
QPS = 41340.01322705692
0.91584
QPS = 41456.67034352971
0.91584
QPS = 42736.137234803486
